# 0 IMPORTS

In [1]:
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
from IPython.core.display import HTML

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


## 0.1. FUNCTIONS

In [2]:
def jupyter_settings():
    %matplotlib inline
    
    plt.style.use( 'ggplot')
    plt.rcParams['figure.figsize'] = [24, 9]
    plt.rcParams['font.size'] = 24
    
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    
    sns.set()
    
jupyter_settings()

def describe_num(num_attributes):
    a = num_attributes.describe().T.reset_index()
    
    d4 = pd.DataFrame(num_attributes.apply( lambda x: x.max() - x.min())).T
    d5 = pd.DataFrame(num_attributes.apply( lambda x: x.skew())).T      
    d6 = pd.DataFrame(num_attributes.apply( lambda x: x.kurtosis())).T
    d7 = pd.DataFrame(num_attributes.apply( lambda x: x.unique().shape[0])).T
    
    n = pd.concat([d4, d5, d6, d7], axis=0).T.reset_index()
    n.columns = ['attibutes', 'range', 'skew', 'kurtosis', 'unique']
    b = a.merge(n, right_on='attibutes', left_on='index')
    b.drop('index', axis=1, inplace=True)
    columns = ['attibutes', 'count', 'mean', 'std', '25%', '50%', '75%', 'min', 'max', 'range',
           'skew', 'kurtosis', 'unique']
    return b[columns]

pd.options.display.float_format = '{:.2f}'.format

## 0.2. IMPORTS DATA

### 0.2.0 ABA - AMAZONIA LEGAL

In [3]:
# IBGE municipios AM Legal
am_legal = pd.read_excel('../data/MUNIC/lista_de_municipios_Amazonia_Legal_2020.xls', sheet_name='Planilha1')
am_legal.drop(columns=['Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15'], inplace=True)

# Aba com municipios
municipios_brasil = pd.read_excel('../data/MUNIC/Base_MUNIC_2020.xlsx', sheet_name='Variáveis externa')

am_legal = am_legal[['CD_MUN']]
am_legal['AM_LEGAL'] = 1

df_amz_legal = pd.merge(municipios_brasil, am_legal, left_on='CodMun', right_on='CD_MUN', how='left')
df_amz_legal = df_amz_legal.fillna(0)
df_amz_legal = df_amz_legal.drop(columns=['Regiao', 'UF', 'Cod UF', 'Mun', 'Populacao', 'Faixa_pop', 'CD_MUN'])
df_amz_legal.head()

,CodMun,AM_LEGAL
0,1100015,1.00
1,1100023,1.00
2,1100031,1.00
3,1100049,1.00
4,1100056,1.00


### 0.2.1 ABA - RECURSOS HUMANOS
- Analisar a composição da administração pública

In [4]:
# Leitura dos dados da base fornecida
dfr_munic_rh = pd.read_excel('../data/MUNIC/Base_MUNIC_2020.xlsx', sheet_name='Recursos humanos')
# renomeando para melhor entendimento
map_df = {'Mreh0111': '1_1_rh_qd_pess_dir_estatuarios',
          'Mreh0112': '1_1_rh_qd_pess_dir_celetistas',
          'Mreh0113': '1_1_rh_qd_pess_dir_so_comissionados',
          'Mreh0114': '1_1_rh_qd_pess_dir_estagiarios',
          'Mreh0115': '1_1_rh_qd_pess_dir_sem_vinculo',
          'Mreh0116': '1_1_rh_qd_pess_dir_total',
          'Mreh02'  : '1_2_rh_qd_pess_ind_adm_ind_exist',
          'Mreh0311': '1_2_rh_qd_pess_ind_estatuarios',
          'Mreh0312': '1_2_rh_qd_pess_ind_celetistas',
          'Mreh0313': '1_2_rh_qd_pess_ind_so_comissionados',
          'Mreh0314': '1_2_rh_qd_pess_ind_estagiarios',
          'Mreh0315': '1_2_rh_qd_pess_ind_sem_vinculo',
          'Mreh0316': '1_2_rh_qd_pess_ind_total'}
dfr_munic_rh = dfr_munic_rh.rename(columns=map_df)
dfr_munic_rh = pd.merge(dfr_munic_rh, df_amz_legal, on='CodMun', how='left')

In [5]:
#dfr_munic_rh.to_csv('../dados/munic/2020_munic_recursos_humanos.csv', index=False)

In [6]:
# ajustando nome dos campos para utilização da função agg
map_str = {'Recusa': 0, 'Não informou': 0, '(*) Não soube informar':0, '-':0}
dfr_munic_rh = dfr_munic_rh.replace({'1_1_rh_qd_pess_dir_total': map_str})
dfr_munic_rh = dfr_munic_rh.replace({'1_2_rh_qd_pess_ind_total': map_str})

In [7]:
dfr_munic_rh.head()

,CodMun,UF,Cod UF,Mun,Faixa_pop,Regiao,1_1_rh_qd_pess_dir_estatuarios,1_1_rh_qd_pess_dir_celetistas,1_1_rh_qd_pess_dir_so_comissionados,1_1_rh_qd_pess_dir_estagiarios,1_1_rh_qd_pess_dir_sem_vinculo,1_1_rh_qd_pess_dir_total,1_2_rh_qd_pess_ind_adm_ind_exist,1_2_rh_qd_pess_ind_estatuarios,1_2_rh_qd_pess_ind_celetistas,1_2_rh_qd_pess_ind_so_comissionados,1_2_rh_qd_pess_ind_estagiarios,1_2_rh_qd_pess_ind_sem_vinculo,1_2_rh_qd_pess_ind_total,AM_LEGAL
0,1100015,RO,11,Alta Floresta DOeste,4 - 20001 até 50000,1 - Norte,899,19,47,0,0,965,Sim,0,12,2,0,0,14,1.00
1,1100023,RO,11,Ariquemes,6 - 100001 até 500000,1 - Norte,2260,194,213,76,54,2797,Sim,6,0,4,0,0,10,1.00
2,1100031,RO,11,Cabixi,2 - 5001 até 10000,1 - Norte,271,24,54,0,14,363,Não,-,-,-,-,-,0,1.00
3,1100049,RO,11,Cacoal,5 - 50001 até 100000,1 - Norte,1698,10,222,0,27,1957,Sim,99,0,44,0,0,143,1.00
4,1100056,RO,11,Cerejeiras,3 - 10001 até 20000,1 - Norte,530,35,53,1,31,650,Não,-,-,-,-,-,0,1.00


In [35]:
# analisando a quantidade de pessoas alocadas na administração pública

display(dfr_munic_rh[['AM_LEGAL', '1_1_rh_qd_pess_dir_total', '1_2_rh_qd_pess_ind_total']].groupby('AM_LEGAL').agg({'1_1_rh_qd_pess_dir_total':['count', 'mean'],
                                                                                                        '1_2_rh_qd_pess_ind_total':['count', 'mean']}))
display(dfr_munic_rh[['Regiao', '1_1_rh_qd_pess_dir_total', '1_2_rh_qd_pess_ind_total']].groupby('Regiao').agg({'1_1_rh_qd_pess_dir_total':['count', 'mean'],
                                                                                                        '1_2_rh_qd_pess_ind_total':['mean']}))

1_1_rh_qd_pess_dir_total         1_2_rh_qd_pess_ind_total      
                            count    mean                    count  mean
AM_LEGAL                                                                
0.00                         4798 1091.19                     4798 53.61
1.00                          772 1160.28                      772 36.76

1_1_rh_qd_pess_dir_total         1_2_rh_qd_pess_ind_total
                                    count    mean                     mean
Regiao                                                                    
1 - Norte                             450 1267.61                    17.04
2 - Nordeste                         1794 1111.03                    41.80
3 - Sudeste                          1668 1319.32                    78.29
4 - Sul                              1191  703.46                    35.32
5 - Centro-Oeste                      467 1133.18                    64.85

**Conclusões:** de maneira geral a quantidade média de funcionários lotados nas prefeituras são parecidos com exceção do sul do país com quase metade do valor.

### 0.2.2 ABA - HABITAÇÃO

In [10]:
# Leitura dos dados da base fornecida
dfr_munic_hab = pd.read_excel('../data/MUNIC/Base_MUNIC_2020.xlsx', sheet_name='Habitação')
map_df = {'Mhab01'  : '2_1_hab_orgao_gestor_caract',
          'Mhab03'  : '2_1_hab_orgao_gestor_sexo_tit',
          'Mhab04'  : '2_1_hab_orgao_gestor_idade_tit',
          'Mhab05'  : '2_1_hab_orgao_gestor_cor_tit',
          'Mhab051' : '2_1_hab_orgao_gestor_resp_proprio_tit',
          'Mhab06'  : '2_1_hab_orgao_gestor_escola_tit',
          'Mhab07'  : '2_1_hab_plan_municp_hab_existencia',
          'Mhab071' : '2_1_hab_plan_municp_hab_arct_pde',
          'Mhab081' : '2_1_hab_plan_municp_hab_integ_acoes_c_demais_pol_urb_soc',
          'Mhab082' : '2_1_hab_plan_municp_hab_aprov_infraestrutura',
          'Mhab083' : '2_1_hab_plan_municp_hab_prov_urb_recup_favela_reg_fund',
          'Mhab084' : '2_1_hab_plan_municp_hab_garantir_remocao_area_de_risco',
          'Mhab085' : '2_1_hab_plan_municp_hab_coibir_nv_ocup_inadeq',
          'Mhab086' : '2_1_hab_plan_municp_hab_prod_nv_und_his',
          'Mhab087' : '2_1_hab_plan_municp_hab_otim_rec_fed_est_mun',
          'Mhab088' : '2_1_hab_plan_municp_hab_prior_area_risco',
          'Mhab089' : '2_1_hab_plan_municp_hab_outros',
          'Mhab09'  : '2_1_hab_plan_municp_hab_conf_hab_4anos',
          'Mhab091' : '2_1_hab_plan_municp_hab_conf_hab_ref_pde',
          'Mhab10'  : '2_2_hab_consel_munic_hab_existencia',
          'Mhab101b': '2_2_hab_consel_munic_hab_ano_cria',
          'Mhab102' : '2_2_hab_consel_munic_hab_formacao',
          'Mhab111' : '2_2_hab_consel_munic_hab_conselho_consultivo',
          'Mhab112' : '2_2_hab_consel_munic_hab_conselho_deliberativo',
          'Mhab113' : '2_2_hab_consel_munic_hab_conselho_normativo',
          'Mhab114' : '2_2_hab_consel_munic_hab_conselho_fiscalizador',
          'Mhab12'  : '2_2_hab_consel_munic_hab_qtd_reuniao_12meses',
          'Mhab13'  : '2_2_hab_consel_munic_hab_numero_conselheiros',
          'Mhab141' : '2_2_hab_consel_munic_hab_cap_membros_period',
          'Mhab142' : '2_2_hab_consel_munic_hab_cap_membros_ocasin',
          'Mhab143' : '2_2_hab_consel_munic_hab_cap_membros_nrealiza',
          'Mhab15'  : '2_2_hab_consel_munic_hab_infra_exist',
          'Mhab1511': '2_2_hab_consel_munic_hab_infra_sala',
          'Mhab1512': '2_2_hab_consel_munic_hab_infra_compu',
          'Mhab1513': '2_2_hab_consel_munic_hab_infra_impres',
          'Mhab1514': '2_2_hab_consel_munic_hab_infra_intern',
          'Mhab1515': '2_2_hab_consel_munic_hab_infra_veicu',
          'Mhab1516': '2_2_hab_consel_munic_hab_infra_telef',
          'Mhab1517': '2_2_hab_consel_munic_hab_infra_diarias',
          'Mhab1518': '2_2_hab_consel_munic_hab_infra_dot_orc_propria',
          'Mhab16'  : '2_2_hab_fund_munic_hab_existencia',
          'Mhab161' : '2_2_hab_fund_munic_hab_gestor_pert_cons_m_hab',
          'Mhab17'  : '2_2_hab_fund_munic_hab_reune_recursos',
          'Mhab18'  : '2_3_hab_cad_prog_hab_existencia',
          'Mhab181' : '2_3_hab_cad_prog_hab_ano_realiz',
          'Mhab182' : '2_3_hab_cad_prog_hab_cadast_inform',
          'Mhab183' : '2_3_hab_cad_prog_hab_pretend_familia',
          'Mhab191' : '2_4_hab_hab_irreg_favelas',
          'Mhab192' : '2_4_hab_hab_irreg_corticos',
          'Mhab193' : '2_4_hab_hab_irreg_loteamento',
          'Mhab194' : '2_4_hab_hab_irreg_ocupa_terrenos',
          'Mhab195' : '2_4_hab_hab_irreg_nenhum',
          'Mhab201' : '2_5_hab_prog_hab_18_19_constr_hab',
          'Mhab202' : '2_5_hab_prog_hab_18_19_aquisicao_hab',
          'Mhab203' : '2_5_hab_prog_hab_18_19_melhoria_hab',
          'Mhab204' : '2_5_hab_prog_hab_18_19_oferta_mat_const',
          'Mhab205' : '2_5_hab_prog_hab_18_19_oferta_lotes',
          'Mhab206' : '2_5_hab_prog_hab_18_19_reg_fundiaria',
          'Mhab207' : '2_5_hab_prog_hab_18_19_urb_assent',
          'Mhab208' : '2_5_hab_prog_hab_18_19_nenhum',
          'Mhab21'  : '2_5_hab_prog_hab_18_19_aluguel_social'
         }
dfr_munic_hab = dfr_munic_hab.rename(columns=map_df)
dfr_munic_hab = pd.merge(dfr_munic_hab, df_amz_legal, on='CodMun', how='left')

In [11]:
#dfr_munic_hab.to_csv('../dados/munic/2020_munic_habitacao.csv', index=False)

In [12]:
dfr_munic_hab.head(1)

,CodMun,UF,Cod UF,Mun,Faixa_pop,Regiao,2_1_hab_orgao_gestor_caract,2_1_hab_orgao_gestor_sexo_tit,2_1_hab_orgao_gestor_idade_tit,2_1_hab_orgao_gestor_cor_tit,2_1_hab_orgao_gestor_resp_proprio_tit,2_1_hab_orgao_gestor_escola_tit,2_1_hab_plan_municp_hab_existencia,2_1_hab_plan_municp_hab_arct_pde,2_1_hab_plan_municp_hab_integ_acoes_c_demais_pol_urb_soc,2_1_hab_plan_municp_hab_aprov_infraestrutura,2_1_hab_plan_municp_hab_prov_urb_recup_favela_reg_fund,2_1_hab_plan_municp_hab_garantir_remocao_area_de_risco,2_1_hab_plan_municp_hab_coibir_nv_ocup_inadeq,2_1_hab_plan_municp_hab_prod_nv_und_his,2_1_hab_plan_municp_hab_otim_rec_fed_est_mun,2_1_hab_plan_municp_hab_prior_area_risco,2_1_hab_plan_municp_hab_outros,2_1_hab_plan_municp_hab_conf_hab_4anos,2_1_hab_plan_municp_hab_conf_hab_ref_pde,2_2_hab_consel_munic_hab_existencia,2_2_hab_consel_munic_hab_ano_cria,2_2_hab_consel_munic_hab_formacao,2_2_hab_consel_munic_hab_conselho_consultivo,2_2_hab_consel_munic_hab_conselho_deliberativo,2_2_hab_consel_munic_hab_conselho_normativo,2_2_hab_consel_munic_hab_conselho_fiscalizador,2_2_hab_consel_munic_hab_qtd_reuniao_12meses,2_2_hab_consel_munic_hab_numero_conselheiros,2_2_hab_consel_munic_hab_cap_membros_period,2_2_hab_consel_munic_hab_cap_membros_ocasin,2_2_hab_consel_munic_hab_cap_membros_nrealiza,2_2_hab_consel_munic_hab_infra_exist,2_2_hab_consel_munic_hab_infra_sala,2_2_hab_consel_munic_hab_infra_compu,2_2_hab_consel_munic_hab_infra_impres,2_2_hab_consel_munic_hab_infra_intern,2_2_hab_consel_munic_hab_infra_veicu,2_2_hab_consel_munic_hab_infra_telef,2_2_hab_consel_munic_hab_infra_diarias,2_2_hab_consel_munic_hab_infra_dot_orc_propria,2_2_hab_fund_munic_hab_existencia,2_2_hab_fund_munic_hab_gestor_pert_cons_m_hab,2_2_hab_fund_munic_hab_reune_recursos,2_3_hab_cad_prog_hab_existencia,2_3_hab_cad_prog_hab_ano_realiz,2_3_hab_cad_prog_hab_cadast_inform,2_3_hab_cad_prog_hab_pretend_familia,2_4_hab_hab_irreg_favelas,2_4_hab_hab_irreg_corticos,2_4_hab_hab_irreg_loteamento,2_4_hab_hab_irreg_ocupa_terrenos,2_4_hab_hab_irreg_nenhum,2_5_hab_prog_hab_18_19_constr_hab,2_5_hab_prog_hab_18_19_aquisicao_hab,2_5_hab_prog_hab_18_19_melhoria_hab,2_5_hab_prog_hab_18_19_oferta_mat_const,2_5_hab_prog_hab_18_19_oferta_lotes,2_5_hab_prog_hab_18_19_reg_fundiaria,2_5_hab_prog_hab_18_19_urb_assent,2_5_hab_prog_hab_18_19_nenhum,2_5_hab_prog_hab_18_19_aluguel_social,AM_LEGAL
0,1100015,RO,11,Alta Floresta DOeste,4 - 20001 até 50000,1 - Norte,Não possui estrutura,-,-,-,-,-,Sim,Não,Sim,Sim,Sim,Sim,Não,Sim,Sim,Não,Não,Não,-,Sim,2008,Paritário,Sim,Sim,Não,Sim,0,11,Não,Não,Sim,Não,-,-,-,-,-,-,-,-,Sim,Sim,Não,Sim,2019,Não,Não,Não,Não,Não,Não,Sim,Não,Não,Não,Não,Não,Não,Não,Sim,Sim,1.00


#### análise programa habitacional _ cadastro familia
- Em termos proporcionais a divisão da existência de programa habitacional é parecido na região da AM Legal em relação ao restante do país. Vale ressaltar que uma secretaria exclusiva para este assunto é superior a média nacional na AM Legal

In [66]:
fam = dfr_munic_hab[dfr_munic_hab['2_3_hab_cad_prog_hab_existencia'] == 'Sim']
df_fam = fam[['AM_LEGAL','2_1_hab_orgao_gestor_caract', '2_3_hab_cad_prog_hab_existencia']].groupby(['AM_LEGAL','2_1_hab_orgao_gestor_caract', '2_3_hab_cad_prog_hab_existencia']).size().reset_index()

df_am = df_fam[df_fam['AM_LEGAL'] == 1][['2_1_hab_orgao_gestor_caract', 0]]
df_nam = df_fam[df_fam['AM_LEGAL'] == 0][['2_1_hab_orgao_gestor_caract', 0]]
df_fam = df_fam[['2_1_hab_orgao_gestor_caract', '2_3_hab_cad_prog_hab_existencia']].drop_duplicates().reset_index()
df_fam = pd.merge(df_fam, df_am, on='2_1_hab_orgao_gestor_caract', how='left').rename(columns={0:'AM_LEGAL'})
df_fam['AM_LEGAL_PCT'] = df_fam['AM_LEGAL'] / dfr_munic_hab['AM_LEGAL'].sum() *100
df_fam = pd.merge(df_fam, df_nam, on='2_1_hab_orgao_gestor_caract', how='left').rename(columns={0:'RESTO_PAIS'})
df_fam['RESTO_PAIS_PCT'] = df_fam['RESTO_PAIS'] / (len(dfr_munic_hab) - dfr_munic_hab['AM_LEGAL'].sum()) *100
df_fam

,index,2_1_hab_orgao_gestor_caract,2_3_hab_cad_prog_hab_existencia,AM_LEGAL,AM_LEGAL_PCT,RESTO_PAIS,RESTO_PAIS_PCT
0,0,Não possui estrutura,Sim,123,15.93,861,17.94
1,1,Secretaria em conjunto com outras políticas se...,Sim,94,12.18,615,12.82
2,2,Secretaria exclusiva,Sim,32,4.15,165,3.44
3,3,Setor subordinado a outra secretaria,Sim,220,28.50,1374,28.64
4,4,Setor subordinado diretamente à chefia do Exec...,Sim,13,1.68,141,2.94
5,5,Órgão da administração indireta,Sim,4,0.52,25,0.52


In [37]:

# '2_4_hab_hab_irreg_favelas', '2_4_hab_hab_irreg_corticos', '2_4_hab_hab_irreg_loteamento', '2_4_hab_hab_irreg_ocupa_terrenos', '2_4_hab_hab_irreg_nenhum'
#dfr_munic_hab[['AM_LEGAL','2_4_hab_hab_irreg_nenhum']].groupby(['AM_LEGAL', '2_4_hab_hab_irreg_nenhum']).size().reset_index()

#'2_5_hab_prog_hab_18_19_constr_hab','2_5_hab_prog_hab_18_19_aquisicao_hab','2_5_hab_prog_hab_18_19_melhoria_hab',
#'2_5_hab_prog_hab_18_19_oferta_mat_const','2_5_hab_prog_hab_18_19_oferta_lotes','2_5_hab_prog_hab_18_19_reg_fundiaria',
#'2_5_hab_prog_hab_18_19_urb_assent','2_5_hab_prog_hab_18_19_nenhum','2_5_hab_prog_hab_18_19_aluguel_social'
variavel = '2_4_hab_hab_irreg_favelas'
a = dfr_munic_hab[dfr_munic_hab[variavel]=='Sim'].copy()
display(a[['AM_LEGAL',variavel]].groupby(['AM_LEGAL', variavel]).size().reset_index())
display(a[['Regiao',variavel]].groupby(['Regiao', variavel]).size().reset_index())

,AM_LEGAL,2_4_hab_hab_irreg_favelas,0
0,0.00,Sim,886
1,1.00,Sim,163


,Regiao,2_4_hab_hab_irreg_favelas,0
0,1 - Norte,Sim,131
1,2 - Nordeste,Sim,376
2,3 - Sudeste,Sim,267
3,4 - Sul,Sim,241
4,5 - Centro-Oeste,Sim,34


In [118]:
def analise_brasil(df, variaveis):
    df_base = pd.DataFrame(['AM_LEGAL','1 - Norte','2 - Nordeste','3 - Sudeste','4 - Sul','5 - Centro-Oeste'], columns=['Regiao'])
    for variavel in variaveis:
        a = df[df[variavel]=='Sim'].copy()
        a_ = a[['AM_LEGAL',variavel]].groupby(['AM_LEGAL', variavel]).size().reset_index()
        b = pd.concat([a_[a_['AM_LEGAL'] == 1].reset_index(), a[['Regiao',variavel]].groupby(['Regiao', variavel]).size().reset_index()], axis=0)
        b = b.drop(columns=['index', 'AM_LEGAL', variavel]).fillna('AM_LEGAL')[['Regiao', 0]]
    
        df_base = pd.merge(df_base, b, on='Regiao', how='left')
    
    variaveis.insert(0, 'Regiao')
    df_base.columns = variaveis
    df_base_ = df_base.copy().T
    df_base_.columns = ['AM_LEGAL','1 - Norte','2 - Nordeste','3 - Sudeste','4 - Sul','5 - Centro-Oeste']
    df_base_.drop(index=df_base_.index[0], axis=0, inplace=True)
    return df_base_

In [119]:
variaveis = ['2_4_hab_hab_irreg_favelas', '2_4_hab_hab_irreg_corticos', '2_4_hab_hab_irreg_loteamento', '2_4_hab_hab_irreg_ocupa_terrenos', '2_4_hab_hab_irreg_nenhum']
analise_brasil(dfr_munic_hab, variaveis)

C:\Users\kaotc\AppData\Local\Temp\ipykernel_23276\4010902175.py:9: FutureWarning: Passing 'suffixes' which cause duplicate columns {'0_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_base = pd.merge(df_base, b, on='Regiao', how='left')


,AM_LEGAL,1 - Norte,2 - Nordeste,3 - Sudeste,4 - Sul,5 - Centro-Oeste
2_4_hab_hab_irreg_favelas,163,131,376,267,241,34
2_4_hab_hab_irreg_corticos,125,83,306,313,109,54
2_4_hab_hab_irreg_loteamento,493,312,1030,1212,857,298
2_4_hab_hab_irreg_ocupa_terrenos,126,81,349,195,157,77
2_4_hab_hab_irreg_nenhum,159,75,564,395,289,144


### 0.2.3 ABA - TRANSPORTE

In [15]:
# Leitura dos dados da base fornecida
dfr_munic_transp = pd.read_excel('../data/MUNIC/Base_MUNIC_2020.xlsx', sheet_name='Transporte')
map_df = {'Mtra01'  : "3_1_transp_plan_munic_caract_orgao",
          'Mtra03'  : "3_1_transp_plan_munic_sexo_tit",
          'Mtra04'  : "3_1_transp_plan_munic_idade_tit",
          'Mtra05'  : "3_1_transp_plan_munic_raca_tit",
          'Mtra051' : "3_1_transp_plan_munic_resp_prop_tit",
          'Mtra06'  : "3_1_transp_plan_munic_escolar_tit",
          'Mtra07'  : "3_1_transp_plan_munic_existencia",
          'Mtra081' : "3_1_transp_plan_munic_plt_circ_viaria",
          'Mtra082' : "3_1_transp_plan_munic_organ_sistema",
          'Mtra083' : "3_1_transp_plan_munic_politic_tarifaria",
          'Mtra084' : "3_1_transp_plan_munic_normas_sistema",
          'Mtra085' : "3_1_transp_plan_munic_uso_ped_ciclista",
          'Mtra086' : "3_1_transp_plan_munic_uso_deficiente",
          'Mtra087' : "3_1_transp_plan_munic_outros",
          'Mtra09'  : "3_1_transp_plan_munic_conf_transporte4anos",
          'Mtra091' : "3_1_transp_plan_munic_considerado_plan_mun",
          'Mtra10'  : "3_2_transp_cons_munic_existencia",
          'Mtra101b': "3_2_transp_cons_munic_ano_cria",
          'Mtra102' : "3_2_transp_cons_munic_formacao",
          'Mtra111' : "3_2_transp_cons_munic_conselho_consultivo",
          'Mtra112' : "3_2_transp_cons_munic_conselho_deliberativo",
          'Mtra113' : "3_2_transp_cons_munic_conselho_normativo",
          'Mtra114' : "3_2_transp_cons_munic_conselho_fiscalizador",
          'Mtra12'  : "3_2_transp_cons_munic_qtd_reuniao_12meses",
          'Mtra13'  : "3_2_transp_cons_munic_numero_conselheiros",
          'Mtra141' : "3_2_transp_cons_munic_cap_membros_period",
          'Mtra142' : "3_2_transp_cons_munic_cap_membros_ocasin",
          'Mtra143' : "3_2_transp_cons_munic_cap_membros_nrealiza",
          'Mtra15'  : "3_2_transp_cons_munic_infra_exist",
          'Mtra1511': "3_2_transp_cons_munic_infra_sala",
          'Mtra1512': "3_2_transp_cons_munic_infra_compu",
          'Mtra1513': "3_2_transp_cons_munic_infra_impres",
          'Mtra1514': "3_2_transp_cons_munic_infra_intern",
          'Mtra1515': "3_2_transp_cons_munic_infra_veicu",
          'Mtra1516': "3_2_transp_cons_munic_infra_telef",
          'Mtra1517': "3_2_transp_cons_munic_infra_diarias",
          'Mtra1518': "3_2_transp_cons_munic_infra_dot_orc_propria",
          'Mtra16'  : "3_2_transp_fund_munic_existencia",
          'Mtra161' : "3_2_transp_fund_munic_gestor_pert_cons_m_trans",
          'Mtra17'  : "3_2_transp_fund_munic_financiado_acoes",
          'Mtra181' : "3_3_transp_exist_barco",
          'Mtra182' : "3_3_transp_exist_metro",
          'Mtra183' : "3_3_transp_exist_mototaxi",
          'Mtra184' : "3_3_transp_exist_taxi",
          'Mtra185' : "3_3_transp_exist_trem",
          'Mtra186' : "3_3_transp_exist_van",
          'Mtra187' : "3_3_transp_exist_aviao",
          'Mtra188' : "3_3_transp_exist_app",
          'Mtra189' : "3_3_transp_exist_nenhum",
          'Mtra19'  : "3_3_transp_onibus_existencia",
          'Mtra1911': "3_3_transp_onibus_concessao",
          'Mtra192' : "3_3_transp_onibus_concessao_licit",
          'Mtra1912': "3_3_transp_onibus_permissao",
          'Mtra193' : "3_3_transp_onibus_permissao_licit",
          'Mtra1913': "3_3_transp_onibus_autorizacao",
          'Mtra1914': "3_3_transp_onibus_serv_by_pref",
          'Mtra1915': "3_3_transp_onibus_nao_regulament",
          'Mtra201' : "3_3_transp_onibus_isencao_idoso",
          'Mtra202' : "3_3_transp_onibus_isencao_est_publica",
          'Mtra203' : "3_3_transp_onibus_isencao_est_partic",
          'Mtra204' : "3_3_transp_onibus_isencao_carteiros",
          'Mtra205' : "3_3_transp_onibus_isencao_pcd",
          'Mtra206' : "3_3_transp_onibus_isencao_policiais",
          'Mtra207' : "3_3_transp_onibus_isencao_professor",
          'Mtra208' : "3_3_transp_onibus_isencao_crianca_5anos",
          'Mtra209' : "3_3_transp_onibus_isencao_outros",
          'Mtra2010': "3_3_transp_onibus_isencao_toda_populacao",
          'Mtra2011': "3_3_transp_onibus_isencao_nenhum_passag",
          'Mtra21'  : "3_3_transp_onibus_pcd_existencia",
          'Mtra221' : "3_3_transp_onibus_pcd_piso_baixo",
          'Mtra222' : "3_3_transp_onibus_pcd_plataforma",
          'Mtra223' : "3_3_transp_onibus_pcd_elevatoria",
          'Mtra224' : "3_3_transp_onibus_pcd_nao_sabe",
          'Mtra23'  : "3_3_transp_intermunic_existencia",
          'Mtra231' : "3_3_transp_intermunic_atende_municip",
          'Mtra24'  : "3_3_transp_ciclovia",
          'Mtra25'  : "3_3_transp_bicicletario"
         }
dfr_munic_transp = dfr_munic_transp.rename(columns=map_df)
dfr_munic_transp = pd.merge(dfr_munic_transp, df_amz_legal, on='CodMun', how='left')

In [16]:
#dfr_munic_transp.to_csv('../dados/munic/2020_munic_transporte.csv', index=False)

In [17]:
dfr_munic_transp.head(1)

,CodMun,UF,Cod UF,Mun,Faixa_pop,Regiao,3_1_transp_plan_munic_caract_orgao,3_1_transp_plan_munic_sexo_tit,3_1_transp_plan_munic_idade_tit,3_1_transp_plan_munic_raca_tit,3_1_transp_plan_munic_resp_prop_tit,3_1_transp_plan_munic_escolar_tit,3_1_transp_plan_munic_existencia,3_1_transp_plan_munic_plt_circ_viaria,3_1_transp_plan_munic_organ_sistema,3_1_transp_plan_munic_politic_tarifaria,3_1_transp_plan_munic_normas_sistema,3_1_transp_plan_munic_uso_ped_ciclista,3_1_transp_plan_munic_uso_deficiente,3_1_transp_plan_munic_outros,3_1_transp_plan_munic_conf_transporte4anos,3_1_transp_plan_munic_considerado_plan_mun,3_2_transp_cons_munic_existencia,3_2_transp_cons_munic_ano_cria,3_2_transp_cons_munic_formacao,3_2_transp_cons_munic_conselho_consultivo,3_2_transp_cons_munic_conselho_deliberativo,3_2_transp_cons_munic_conselho_normativo,3_2_transp_cons_munic_conselho_fiscalizador,3_2_transp_cons_munic_qtd_reuniao_12meses,3_2_transp_cons_munic_numero_conselheiros,3_2_transp_cons_munic_cap_membros_period,3_2_transp_cons_munic_cap_membros_ocasin,3_2_transp_cons_munic_cap_membros_nrealiza,3_2_transp_cons_munic_infra_exist,3_2_transp_cons_munic_infra_sala,3_2_transp_cons_munic_infra_compu,3_2_transp_cons_munic_infra_impres,3_2_transp_cons_munic_infra_intern,3_2_transp_cons_munic_infra_veicu,3_2_transp_cons_munic_infra_telef,3_2_transp_cons_munic_infra_diarias,3_2_transp_cons_munic_infra_dot_orc_propria,3_2_transp_fund_munic_existencia,3_2_transp_fund_munic_gestor_pert_cons_m_trans,3_2_transp_fund_munic_financiado_acoes,3_3_transp_exist_barco,3_3_transp_exist_metro,3_3_transp_exist_mototaxi,3_3_transp_exist_taxi,3_3_transp_exist_trem,3_3_transp_exist_van,3_3_transp_exist_aviao,3_3_transp_exist_app,3_3_transp_exist_nenhum,3_3_transp_onibus_existencia,3_3_transp_onibus_concessao,3_3_transp_onibus_concessao_licit,3_3_transp_onibus_permissao,3_3_transp_onibus_permissao_licit,3_3_transp_onibus_autorizacao,3_3_transp_onibus_serv_by_pref,3_3_transp_onibus_nao_regulament,3_3_transp_onibus_isencao_idoso,3_3_transp_onibus_isencao_est_publica,3_3_transp_onibus_isencao_est_partic,3_3_transp_onibus_isencao_carteiros,3_3_transp_onibus_isencao_pcd,3_3_transp_onibus_isencao_policiais,3_3_transp_onibus_isencao_professor,3_3_transp_onibus_isencao_crianca_5anos,3_3_transp_onibus_isencao_outros,3_3_transp_onibus_isencao_toda_populacao,3_3_transp_onibus_isencao_nenhum_passag,3_3_transp_onibus_pcd_existencia,3_3_transp_onibus_pcd_piso_baixo,3_3_transp_onibus_pcd_plataforma,3_3_transp_onibus_pcd_elevatoria,3_3_transp_onibus_pcd_nao_sabe,3_3_transp_intermunic_existencia,3_3_transp_intermunic_atende_municip,3_3_transp_ciclovia,3_3_transp_bicicletario,AM_LEGAL
0,1100015,RO,11,Alta Floresta DOeste,4 - 20001 até 50000,1 - Norte,Não possui estrutura,-,-,-,-,-,Não,-,-,-,-,-,-,-,Não,-,Não,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,Não,-,-,Não,Não,Sim,Sim,Não,Não,Não,Sim,Não,Não,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,Não,-,Não,Não,1.00


In [18]:
dfr_munic_transp.columns.tolist()

['CodMun',
 'UF',
 'Cod UF',
 'Mun',
 'Faixa_pop',
 'Regiao',
 '3_1_transp_plan_munic_caract_orgao',
 '3_1_transp_plan_munic_sexo_tit',
 '3_1_transp_plan_munic_idade_tit',
 '3_1_transp_plan_munic_raca_tit',
 '3_1_transp_plan_munic_resp_prop_tit',
 '3_1_transp_plan_munic_escolar_tit',
 '3_1_transp_plan_munic_existencia',
 '3_1_transp_plan_munic_plt_circ_viaria',
 '3_1_transp_plan_munic_organ_sistema',
 '3_1_transp_plan_munic_politic_tarifaria',
 '3_1_transp_plan_munic_normas_sistema',
 '3_1_transp_plan_munic_uso_ped_ciclista',
 '3_1_transp_plan_munic_uso_deficiente',
 '3_1_transp_plan_munic_outros',
 '3_1_transp_plan_munic_conf_transporte4anos',
 '3_1_transp_plan_munic_considerado_plan_mun',
 '3_2_transp_cons_munic_existencia',
 '3_2_transp_cons_munic_ano_cria',
 '3_2_transp_cons_munic_formacao',
 '3_2_transp_cons_munic_conselho_consultivo',
 '3_2_transp_cons_munic_conselho_deliberativo',
 '3_2_transp_cons_munic_conselho_normativo',
 '3_2_transp_cons_munic_conselho_fiscalizador',
 '3_2_

In [19]:
# Análise Transporte
#'3_3_transp_exist_barco','3_3_transp_exist_metro','3_3_transp_exist_mototaxi','3_3_transp_exist_taxi',
#'3_3_transp_exist_trem','3_3_transp_exist_van','3_3_transp_exist_aviao','3_3_transp_exist_app','3_3_transp_exist_nenhum',

#'3_3_transp_onibus_existencia',
##'3_3_transp_onibus_concessao','3_3_transp_onibus_concessao_licit',
##'3_3_transp_onibus_permissao','3_3_transp_onibus_permissao_licit',
##'3_3_transp_onibus_autorizacao','3_3_transp_onibus_serv_by_pref','3_3_transp_onibus_nao_regulament',

#'3_3_transp_onibus_isencao_idoso','3_3_transp_onibus_isencao_est_publica','3_3_transp_onibus_isencao_est_partic','3_3_transp_onibus_isencao_carteiros',
#'3_3_transp_onibus_isencao_pcd','3_3_transp_onibus_isencao_policiais','3_3_transp_onibus_isencao_professor','3_3_transp_onibus_isencao_crianca_5anos',
#'3_3_transp_onibus_isencao_outros','3_3_transp_onibus_isencao_toda_populacao','3_3_transp_onibus_isencao_nenhum_passag',

##'3_3_transp_onibus_pcd_existencia',
#'3_3_transp_onibus_pcd_piso_baixo','3_3_transp_onibus_pcd_plataforma','3_3_transp_onibus_pcd_elevatoria','3_3_transp_onibus_pcd_nao_sabe',

#'3_3_transp_intermunic_existencia','3_3_transp_intermunic_atende_municip',

#'3_3_transp_ciclovia','3_3_transp_bicicletario',
variavel = '3_3_transp_bicicletario'
a = dfr_munic_transp[dfr_munic_transp[variavel]=='Sim'].copy()
display(a[['AM_LEGAL',variavel]].groupby(['AM_LEGAL', variavel]).size().reset_index())
display(a[['Regiao',variavel]].groupby(['Regiao', variavel]).size().reset_index())

,AM_LEGAL,3_3_transp_bicicletario,0
0,0.00,Sim,356
1,1.00,Sim,25


,Regiao,3_3_transp_bicicletario,0
0,1 - Norte,Sim,18
1,2 - Nordeste,Sim,48
2,3 - Sudeste,Sim,177
3,4 - Sul,Sim,109
4,5 - Centro-Oeste,Sim,29


### 0.2.4 ABA - AGROPECUARIO

In [20]:
# Leitura dos dados da base fornecida
dfr_munic_agropec = pd.read_excel('../data/MUNIC/Base_MUNIC_2020.xlsx', sheet_name='Agropecuário')
map_df = {'Magr01'   : "4_1_agrop_gest_polit_caract_orgao",
          'Magr03'   : "4_1_agrop_gest_polit_sexo_tit",
          'Magr04'   : "4_1_agrop_gest_polit_idade_tit",
          'Magr05'   : "4_1_agrop_gest_polit_raca_tit",
          'Magr051'  : "4_1_agrop_gest_polit_resp_prop_tit",
          'Magr06'   : "4_1_agrop_gest_polit_escolar_tit",
          'Magr07'   : "4_2_agrop_cons_desen_rural_existencia",
          'Magr071'  : "4_2_agrop_cons_desen_rural_ano_cria",
          'Magr072'  : "4_2_agrop_cons_desen_rural_formacao",
          'Magr081'  : "4_2_agrop_cons_desen_rural_conselho_consultivo",
          'Magr082'  : "4_2_agrop_cons_desen_rural_conselho_deliberativo",
          'Magr083'  : "4_2_agrop_cons_desen_rural_conselho_normativo",
          'Magr084'  : "4_2_agrop_cons_desen_rural_conselho_fiscalizador",
          'Magr09'   : "4_2_agrop_cons_desen_rural_qtd_reuniao_12meses",
          'Magr10'   : "4_2_agrop_cons_desen_rural_numero_conselheiros",
          'Magr111'  : "4_2_agrop_cons_desen_rural_cap_membros_period",
          'Magr112'  : "4_2_agrop_cons_desen_rural_cap_membros_ocasin",
          'Magr113'  : "4_2_agrop_cons_desen_rural_cap_membros_nrealiza",
          'Magr12'   : "4_2_agrop_cons_desen_rural_infra_exist",
          'Magr1211' : "4_2_agrop_cons_desen_rural_infra_sala",
          'Magr1212' : "4_2_agrop_cons_desen_rural_infra_compu",
          'Magr1213' : "4_2_agrop_cons_desen_rural_infra_impres",
          'Magr1214' : "4_2_agrop_cons_desen_rural_infra_intern",
          'Magr1215' : "4_2_agrop_cons_desen_rural_infra_veicu",
          'Magr1216' : "4_2_agrop_cons_desen_rural_infra_telef",
          'Magr1217' : "4_2_agrop_cons_desen_rural_infra_diarias",
          'Magr1218' : "4_2_agrop_cons_desen_rural_infra_dot_orc_propria",
          'Magr131'  : "4_3_agrop_prog_acoes_semetes_existencia",
          'Magr13111': "4_3_agrop_prog_acoes_semetes_dist_gratuita",
          'Magr13112': "4_3_agrop_prog_acoes_semetes_mais_barata",
          'Magr13113': "4_3_agrop_prog_acoes_semetes_financiado",
          'Magr13114': "4_3_agrop_prog_acoes_semetes_outros",
          'Magr132'  : "4_3_agrop_prog_acoes_mudas_existencia",
          'Magr13211': "4_3_agrop_prog_acoes_mudas_dist_gratuita",
          'Magr13212': "4_3_agrop_prog_acoes_mudas_mais_barata",
          'Magr13213': "4_3_agrop_prog_acoes_mudas_financiado",
          'Magr13214': "4_3_agrop_prog_acoes_mudas_outros",
          'Magr133'  : "4_3_agrop_prog_acoes_adubos_existencia",
          'Magr13311': "4_3_agrop_prog_acoes_adubos_dist_gratuita",
          'Magr13312': "4_3_agrop_prog_acoes_adubos_mais_barata",
          'Magr13313': "4_3_agrop_prog_acoes_adubos_financiado",
          'Magr13314': "4_3_agrop_prog_acoes_adubos_outros",
          'Magr134'  : "4_3_agrop_prog_acoes_racao_adub_existencia",
          'Magr13411': "4_3_agrop_prog_acoes_racao_adub_dist_gratuita",
          'Magr13412': "4_3_agrop_prog_acoes_racao_adub_mais_barata",
          'Magr13413': "4_3_agrop_prog_acoes_racao_adub_financiado",
          'Magr13414': "4_3_agrop_prog_acoes_racao_adub_outros",
          'Magr135'  : "4_3_agrop_prog_acoes_alevinos_existencia",
          'Magr13511': "4_3_agrop_prog_acoes_alevinos_dist_gratuita",
          'Magr13512': "4_3_agrop_prog_acoes_alevinos_mais_barata",
          'Magr13513': "4_3_agrop_prog_acoes_alevinos_financiado",
          'Magr13514': "4_3_agrop_prog_acoes_alevinos_outros",
          'Magr136'  : "4_3_agrop_prog_acoes_outros_insumos",
          'Magr141'  : "4_3_agrop_prog_acoes_maquin_cessao_temp_gratu",
          'Magr142'  : "4_3_agrop_prog_acoes_maquin_aluguel",
          'Magr143'  : "4_3_agrop_prog_acoes_maquin_outro",
          'Magr144'  : "4_3_agrop_prog_acoes_maquin_nenhumo",
          'Magr151'  : "4_3_agrop_prog_acoes_agric_organica",
          'Magr152'  : "4_3_agrop_prog_acoes_agric_familiar",
          'Magr153'  : "4_3_agrop_prog_acoes_aquicultura",
          'Magr154'  : "4_3_agrop_prog_acoes_pesca",
          'Magr155'  : "4_3_agrop_prog_acoes_hortas_comunitarias",
          'Magr16'   : "4_4_agrop_promo_festiv_existencia",
          'Magr171'  : "4_4_agrop_promo_festiv_festividades",
          'Magr172'  : "4_4_agrop_promo_festiv_premiacoes",
          'Magr18'   : "4_4_agrop_preven_prob_climatico",
          'Magr191'  : "4_5_agrop_assistec_municipal",
          'Magr192'  : "4_5_agrop_assistec_estadual",
          'Magr193'  : "4_5_agrop_assistec_federal",
          'Magr194'  : "4_5_agrop_assistec_naoexiste",
          'Magr20'   : "4_5_agrop_assistec_inst_conveniadas",
          'Magr211'  : "4_5_agrop_assistec_empres_privada",
          'Magr212'  : "4_5_agrop_assistec_ong",
          'Magr213'  : "4_5_agrop_assistec_outro",
          'Magr214'  : "4_5_agrop_assistec_nenhum",
          'Magr221'  : "4_5_agrop_prog_acoes_social_existencia",
          'Magr22111': "4_5_agrop_prog_acoes_social_educa",
          'Magr22112': "4_5_agrop_prog_acoes_social_saude",
          'Magr22113': "4_5_agrop_prog_acoes_social_alimen",
          'Magr22114': "4_5_agrop_prog_acoes_social_outros",
          'Magr222'  : "4_5_agrop_prog_acoes_artes_existencia",
          'Magr22211': "4_5_agrop_prog_acoes_artes_capacit",
          'Magr22212': "4_5_agrop_prog_acoes_artes_apoio_comer",
          'Magr22213': "4_5_agrop_prog_acoes_artes_associatv",
          'Magr22214': "4_5_agrop_prog_acoes_artes_outros",
          'Magr231'  : "4_6_agrop_associatv_cadpref_sind_rural",
          'Magr232'  : "4_6_agrop_associatv_cadpref_assoc_prod",
          'Magr233'  : "4_6_agrop_associatv_cadpref_cooperat",
          'Magr234'  : "4_6_agrop_associatv_cadpref_naopossui",
          'Magr241'  : "4_6_agrop_associatv_atua_munc_sind_rural",
          'Magr242'  : "4_6_agrop_associatv_atua_munc_assoc_prod",
          'Magr243'  : "4_6_agrop_associatv_atua_munc_cooperat",
          'Magr244'  : "4_6_agrop_associatv_atua_munc_outro",
          'Magr245'  : "4_6_agrop_associatv_atua_munc_nenhuma",
          'Magr251'  : "4_7_agrop_infra_centro_comercial",
          'Magr252'  : "4_7_agrop_infra_feiras",
          'Magr253'  : "4_7_agrop_infra_parque_exposicao",
          'Magr254'  : "4_7_agrop_infra_c_comercial_vegetal",
          'Magr255'  : "4_7_agrop_infra_outro",
          'Magr256'  : "4_7_agrop_infra_nenhum",
          'Magr261'  : "4_8_agrop_aq_prod_direto_prod",
          'Magr262'  : "4_8_agrop_aq_prod_atraves_entid",
          'Magr263'  : "4_8_agrop_aq_prod_prog_aquis",
          'Magr264'  : "4_8_agrop_aq_prod_outro",
          'Magr265'  : "4_8_agrop_aq_prod_nenhum",
          'Magr271'  : "4_8_agrop_vacina_gratuito",
          'Magr272'  : "4_8_agrop_vacina_maisbarato",
          'Magr273'  : "4_8_agrop_vacina_outro",
          'Magr274'  : "4_8_agrop_vacina_nenhum",
          'Magr28'   : "4_8_agrop_impl_sim",
          'Magr29'   : "4_8_agrop_abatedouro_existencia",
          'Magr291'  : "4_8_agrop_abatedouro_uso12meses",
          'Magr30'   : "4_8_agrop_exist_extracao_vegetal"
        }
dfr_munic_agropec = dfr_munic_agropec.rename(columns=map_df)
dfr_munic_agropec = pd.merge(dfr_munic_agropec, df_amz_legal, on='CodMun', how='left')

In [21]:
#dfr_munic_agropec.to_csv('../dados/munic/2020_munic_agropecuaria.csv', index=False)

In [22]:
dfr_munic_agropec[['AM_LEGAL','4_5_agrop_assistec_inst_conveniadas']].groupby(['AM_LEGAL', '4_5_agrop_assistec_inst_conveniadas']).size().reset_index()

,AM_LEGAL,4_5_agrop_assistec_inst_conveniadas,0
0,0.00,Instituições privadas,175
1,0.00,Instituições públicas,2802
2,0.00,Instituições públicas e privadas,835
3,0.00,Não informou,6
4,0.00,Não possui convênio,960
5,0.00,Recusa,20
6,1.00,Instituições privadas,13
7,1.00,Instituições públicas,392
8,1.00,Instituições públicas e privadas,89
9,1.00,Não informou,13


In [23]:
dfr_munic_agropec.columns.tolist()

['CodMun',
 'UF',
 'Cod UF',
 'Mun',
 'Faixa_pop',
 'Regiao',
 '4_1_agrop_gest_polit_caract_orgao',
 '4_1_agrop_gest_polit_sexo_tit',
 '4_1_agrop_gest_polit_idade_tit',
 '4_1_agrop_gest_polit_raca_tit',
 '4_1_agrop_gest_polit_resp_prop_tit',
 '4_1_agrop_gest_polit_escolar_tit',
 '4_2_agrop_cons_desen_rural_existencia',
 '4_2_agrop_cons_desen_rural_ano_cria',
 '4_2_agrop_cons_desen_rural_formacao',
 '4_2_agrop_cons_desen_rural_conselho_consultivo',
 '4_2_agrop_cons_desen_rural_conselho_deliberativo',
 '4_2_agrop_cons_desen_rural_conselho_normativo',
 '4_2_agrop_cons_desen_rural_conselho_fiscalizador',
 '4_2_agrop_cons_desen_rural_qtd_reuniao_12meses',
 '4_2_agrop_cons_desen_rural_numero_conselheiros',
 '4_2_agrop_cons_desen_rural_cap_membros_period',
 '4_2_agrop_cons_desen_rural_cap_membros_ocasin',
 '4_2_agrop_cons_desen_rural_cap_membros_nrealiza',
 '4_2_agrop_cons_desen_rural_infra_exist',
 '4_2_agrop_cons_desen_rural_infra_sala',
 '4_2_agrop_cons_desen_rural_infra_compu',
 '4_2_agro

In [24]:
##'4_3_agrop_prog_acoes_semetes_existencia',
#'4_3_agrop_prog_acoes_semetes_dist_gratuita','4_3_agrop_prog_acoes_semetes_mais_barata','4_3_agrop_prog_acoes_semetes_financiado','4_3_agrop_prog_acoes_semetes_outros',
##'4_3_agrop_prog_acoes_mudas_existencia',
#'4_3_agrop_prog_acoes_mudas_dist_gratuita','4_3_agrop_prog_acoes_mudas_mais_barata','4_3_agrop_prog_acoes_mudas_financiado','4_3_agrop_prog_acoes_mudas_outros',
##'4_3_agrop_prog_acoes_adubos_existencia',
#'4_3_agrop_prog_acoes_adubos_dist_gratuita','4_3_agrop_prog_acoes_adubos_mais_barata','4_3_agrop_prog_acoes_adubos_financiado','4_3_agrop_prog_acoes_adubos_outros',
##'4_3_agrop_prog_acoes_racao_adub_existencia',
#'4_3_agrop_prog_acoes_racao_adub_dist_gratuita','4_3_agrop_prog_acoes_racao_adub_mais_barata','4_3_agrop_prog_acoes_racao_adub_financiado','4_3_agrop_prog_acoes_racao_adub_outros',
##'4_3_agrop_prog_acoes_alevinos_existencia',
#'4_3_agrop_prog_acoes_alevinos_dist_gratuita','4_3_agrop_prog_acoes_alevinos_mais_barata','4_3_agrop_prog_acoes_alevinos_financiado','4_3_agrop_prog_acoes_alevinos_outros',
#'4_3_agrop_prog_acoes_outros_insumos',

#'4_3_agrop_prog_acoes_maquin_cessao_temp_gratu', '4_3_agrop_prog_acoes_maquin_aluguel','4_3_agrop_prog_acoes_maquin_outro',
#'4_3_agrop_prog_acoes_maquin_nenhumo','4_3_agrop_prog_acoes_agric_organica','4_3_agrop_prog_acoes_agric_familiar','4_3_agrop_prog_acoes_aquicultura',
#'4_3_agrop_prog_acoes_pesca','4_3_agrop_prog_acoes_hortas_comunitarias',

#'4_4_agrop_promo_festiv_existencia','4_4_agrop_promo_festiv_festividades','4_4_agrop_promo_festiv_premiacoes','4_4_agrop_preven_prob_climatico'

#'4_5_agrop_assistec_municipal','4_5_agrop_assistec_estadual','4_5_agrop_assistec_federal','4_5_agrop_assistec_naoexiste','4_5_agrop_assistec_inst_conveniadas',
#'4_5_agrop_assistec_empres_privada','4_5_agrop_assistec_ong','4_5_agrop_assistec_outro','4_5_agrop_assistec_nenhum',
##'4_5_agrop_prog_acoes_social_existencia',
#'4_5_agrop_prog_acoes_social_educa','4_5_agrop_prog_acoes_social_saude','4_5_agrop_prog_acoes_social_alimen','4_5_agrop_prog_acoes_social_outros',
##'4_5_agrop_prog_acoes_artes_existencia',
#'4_5_agrop_prog_acoes_artes_capacit','4_5_agrop_prog_acoes_artes_apoio_comer','4_5_agrop_prog_acoes_artes_associatv','4_5_agrop_prog_acoes_artes_outros',

#'4_6_agrop_associatv_cadpref_sind_rural','4_6_agrop_associatv_cadpref_assoc_prod','4_6_agrop_associatv_cadpref_cooperat','4_6_agrop_associatv_cadpref_naopossui',
#'4_6_agrop_associatv_atua_munc_sind_rural','4_6_agrop_associatv_atua_munc_assoc_prod','4_6_agrop_associatv_atua_munc_cooperat','4_6_agrop_associatv_atua_munc_outro','4_6_agrop_associatv_atua_munc_nenhuma',

#'4_7_agrop_infra_centro_comercial','4_7_agrop_infra_feiras','4_7_agrop_infra_parque_exposicao','4_7_agrop_infra_c_comercial_vegetal','4_7_agrop_infra_outro','4_7_agrop_infra_nenhum',

'4_8_agrop_aq_prod_direto_prod','4_8_agrop_aq_prod_atraves_entid','4_8_agrop_aq_prod_prog_aquis','4_8_agrop_aq_prod_outro','4_8_agrop_aq_prod_nenhum',
'4_8_agrop_vacina_gratuito','4_8_agrop_vacina_maisbarato','4_8_agrop_vacina_outro','4_8_agrop_vacina_nenhum',
'4_8_agrop_impl_sim','4_8_agrop_abatedouro_existencia','4_8_agrop_abatedouro_uso12meses','4_8_agrop_exist_extracao_vegetal',

variavel = '4_8_agrop_exist_extracao_vegetal'
a = dfr_munic_agropec[dfr_munic_agropec[variavel]=='Sim'].copy()
display(a[['AM_LEGAL',variavel]].groupby(['AM_LEGAL', variavel]).size().reset_index())
display(a[['Regiao',variavel]].groupby(['Regiao', variavel]).size().reset_index())

,AM_LEGAL,4_8_agrop_exist_extracao_vegetal,0
0,0.00,Sim,405
1,1.00,Sim,110


,Regiao,4_8_agrop_exist_extracao_vegetal,0
0,1 - Norte,Sim,68
1,2 - Nordeste,Sim,169
2,3 - Sudeste,Sim,113
3,4 - Sul,Sim,134
4,5 - Centro-Oeste,Sim,31


### 0.2.5 ABA - MEIO AMBIENTE

In [25]:
# Leitura dos dados da base fornecida
dfr_munic_ambiente = pd.read_excel('../data/MUNIC/Base_MUNIC_2020.xlsx', sheet_name='Meio ambiente')
map_df = {'Mmam01'   : "5_1_ambiente_gest_polit_caract_orgao",
          'Mmam03'   : "5_1_ambiente_gest_polit_sexo_tit",
          'Mmam04'   : "5_1_ambiente_gest_polit_idade_tit",
          'Mmam05'   : "5_1_ambiente_gest_polit_raca_tit",
          'Mmam051'  : "5_1_ambiente_gest_polit_resp_prop_tit",
          'Mmam06'   : "5_1_ambiente_gest_polit_escolar_tit",
          'Mmam08'   : "5_2_ambiente_capac_ult4anos_govfed",
          'Mmam091'  : "5_2_ambiente_capac_area_estrutura",
          'Mmam092'  : "5_2_ambiente_capac_area_licenciam",
          'Mmam093'  : "5_2_ambiente_capac_area_edu_ambiental",
          'Mmam094'  : "5_2_ambiente_capac_area_edu_amb_agric_fam",
          'Mmam095'  : "5_2_ambiente_capac_area_cad_amb_rural",
          'Mmam096'  : "5_2_ambiente_capac_area_resid_solidos",
          'Mmam097'  : "5_2_ambiente_capac_area_prod_cons_sustent",
          'Mmam098'  : "5_2_ambiente_capac_area_mudanca_cimatica",
          'Mmam099'  : "5_2_ambiente_capac_area_recursos_hidr",
          'Mmam0910' : "5_2_ambiente_capac_area_part_foruns_coleg",
          'Mmam0911' : "5_2_ambiente_capac_area_outras",
          'Mmam10'   : "5_3_ambiente_cons_existencia",
          'Mmam101b' : "5_3_ambiente_cons_ano_cria",
          'Mmam102'  : "5_3_ambiente_cons_formacao",
          'Mmam111'  : "5_3_ambiente_cons_conselho_consultivo",
          'Mmam112'  : "5_3_ambiente_cons_conselho_deliberativo",
          'Mmam113'  : "5_3_ambiente_cons_conselho_normativo",
          'Mmam114'  : "5_3_ambiente_cons_conselho_fiscalizador",
          'Mmam12'   : "5_3_ambiente_cons_qtd_reuniao_12meses",
          'Mmam13'   : "5_3_ambiente_cons_numero_conselheiros",
          'Mmam141'  : "5_3_ambiente_cons_cap_membros_period",
          'Mmam142'  : "5_3_ambiente_cons_cap_membros_ocasin",
          'Mmam143'  : "5_3_ambiente_cons_cap_membros_nrealiza",
          'Mmam15'   : "5_3_ambiente_cons_infra_exist",
          'Mmam1511' : "5_3_ambiente_cons_infra_sala",
          'Mmam1512' : "5_3_ambiente_cons_infra_compu",
          'Mmam1513' : "5_3_ambiente_cons_infra_impres",
          'Mmam1514' : "5_3_ambiente_cons_infra_intern",
          'Mmam1515' : "5_3_ambiente_cons_infra_veicu",
          'Mmam1516' : "5_3_ambiente_cons_infra_telef",
          'Mmam1517' : "5_3_ambiente_cons_infra_diarias",
          'Mmam1518' : "5_3_ambiente_cons_infra_dot_orc_propria",
          'Mmam16'   : "5_3_ambiente_possui_rec_proprio",
          'Mmam17'   : "5_3_ambiente_fund_munic_existencia",
          'Mmam171'  : "5_3_ambiente_fund_munic_gestor_fund_cons",
          'Mmam18'   : "5_3_ambiente_fund_munic_2019_empenhado",
          'Mmam201'  : "5_4_ambiente_legis_colet_sel_existencia",
          'Mmam20011': "5_4_ambiente_legis_colet_sel_ano",
          'Mmam202'  : "5_4_ambiente_legis_san_bas_existencia",
          'Mmam2021' : "5_4_ambiente_legis_san_bas_ano",
          'Mmam203'  : "5_4_ambiente_legis_gest_bachidro_existencia",
          'Mmam2031' : "5_4_ambiente_legis_gest_bachidro_ano",
          'Mmam204'  : "5_4_ambiente_legis_z_prot_existencia",
          'Mmam2041' : "5_4_ambiente_legis_z_prot_ano",
          'Mmam205'  : "5_4_ambiente_legis_dest_emb_agrotx_existencia",
          'Mmam2051' : "5_4_ambiente_legis_dest_emb_agrotx_ano",
          'Mmam206'  : "5_4_ambiente_legis_polu_ar_existencia",
          'Mmam2061' : "5_4_ambiente_legis_polu_ar_ano",
          'Mmam207'  : "5_4_ambiente_legis_perm_ext_miner_existencia",
          'Mmam2071' : "5_4_ambiente_legis_perm_ext_miner_ano",
          'Mmam208'  : "5_4_ambiente_legis_fauna_silv_existencia",
          'Mmam2081' : "5_4_ambiente_legis_fauna_silv_ano",
          'Mmam209'  : "5_4_ambiente_legis_floresta_existencia",
          'Mmam2091' : "5_4_ambiente_legis_floresta_ano",
          'Mmam2010' : "5_4_ambiente_legis_prt_biodv_existencia",
          'Mmam20101': "5_4_ambiente_legis_prt_biodv_ano",
          'Mmam2011' : "5_4_ambiente_legis_mit_md_clima_existencia",
          'Mmam20111': "5_4_ambiente_legis_mit_md_clima_ano",
          'Mmam2012' : "5_4_ambiente_legis_nenhuma",
          'Mmam21'   : "5_5_ambiente_plano_res_solido_existencia",
          'Mmam211'  : "5_5_ambiente_plano_res_solido_ap_municip",
          'Mmam221'  : "5_6_ambiente_par_govfed_colet_educador",
          'Mmam222'  : "5_6_ambiente_par_govfed_sala_verde",
          'Mmam223'  : "5_6_ambiente_par_govfed_circ_tela_verde",
          'Mmam224'  : "5_6_ambiente_par_govfed_conf_meioambiente",
          'Mmam225'  : "5_6_ambiente_par_govfed_edu_ambiental_pgirs",
          'Mmam226'  : "5_6_ambiente_par_govfed_sust_ambiental",
          'Mmam227'  : "5_6_ambiente_par_govfed_edu_ambiental_familiar",
          'Mmam228'  : "5_6_ambiente_par_govfed_confer_nacional",
          'Mmam229'  : "5_6_ambiente_par_govfed_nenhum",
          'Mmam23'   : "5_7_ambiente_pag_serv_ambi_fonte_diretamente",
          'Mmam2311' : "5_7_ambiente_pag_serv_ambi_fonte_12m_municp",
          'Mmam2312' : "5_7_ambiente_pag_serv_ambi_fonte_12m_govfed",
          'Mmam2313' : "5_7_ambiente_pag_serv_ambi_fonte_12m_govest",
          'Mmam2314' : "5_7_ambiente_pag_serv_ambi_fonte_12m_inipriv",
          'Mmam2315' : "5_7_ambiente_pag_serv_ambi_fonte_12m_ong",
          'Mmam2316' : "5_7_ambiente_pag_serv_ambi_fonte_12m_doacoes",
          'Mmam2317' : "5_7_ambiente_pag_serv_ambi_fonte_12m_outros",
          'Mmam241'  : "5_7_ambiente_pag_serv_ambi_tipo_rec_con_rec_hidrico",
          'Mmam242'  : "5_7_ambiente_pag_serv_ambi_tipo_rec_con_vegt_nativa",
          'Mmam243'  : "5_7_ambiente_pag_serv_ambi_tipo_rec_con_unid_conserv",
          'Mmam244'  : "5_7_ambiente_pag_serv_ambi_tipo_rec_con_cobert_vegt",
          'Mmam245'  : "5_7_ambiente_pag_serv_ambi_tipo_conserv_vegt_urbana",
          'Mmam246'  : "5_7_ambiente_pag_serv_ambi_tipo_ret_carbono",
          'Mmam247'  : "5_7_ambiente_pag_serv_ambi_tipo_outros",
          'Mmam25'   : "5_8_ambiente_impc_amb_ocorr_24m_existencia",
          'Mmam261'  : "5_8_ambiente_impc_amb_condicoes_extremas",
          'Mmam262'  : "5_8_ambiente_impc_amb_polu_ar",
          'Mmam263'  : "5_8_ambiente_impc_amb_polu_corpo_agua",
          'Mmam264'  : "5_8_ambiente_impc_amb_assoreamento",
          'Mmam265'  : "5_8_ambiente_impc_amb_diminin_vazao",
          'Mmam266'  : "5_8_ambiente_impc_amb_desmatamento",
          'Mmam267'  : "5_8_ambiente_impc_amb_queimadas",
          'Mmam268'  : "5_8_ambiente_impc_amb_contam_solo",
          'Mmam269'  : "5_8_ambiente_impc_amb_erosao_desert",
          'Mmam2610' : "5_8_ambiente_impc_amb_degr_areas_proteg",
          'Mmam2611' : "5_8_ambiente_impc_amb_diminin_biodv",
          'Mmam2612' : "5_8_ambiente_impc_amb_moradia_risco_ambiental",
          'Mmam2613' : "5_8_ambiente_impc_amb_falta_saneamento",
          'Mmam2614' : "5_8_ambiente_impc_amb_outros"
        }
dfr_munic_ambiente = dfr_munic_ambiente.rename(columns=map_df)
dfr_munic_ambiente = pd.merge(dfr_munic_ambiente, df_amz_legal, on='CodMun', how='left')

In [26]:
#dfr_munic_ambiente.to_csv('../dados/munic/2020_munic_meio_ambiente.csv', index=False)

In [27]:
dfr_munic_ambiente.columns.tolist()

['CodMun',
 'UF',
 'Cod UF',
 'Mun',
 'Faixa_pop',
 'Regiao',
 '5_1_ambiente_gest_polit_caract_orgao',
 '5_1_ambiente_gest_polit_sexo_tit',
 '5_1_ambiente_gest_polit_idade_tit',
 '5_1_ambiente_gest_polit_raca_tit',
 '5_1_ambiente_gest_polit_resp_prop_tit',
 '5_1_ambiente_gest_polit_escolar_tit',
 '5_2_ambiente_capac_ult4anos_govfed',
 '5_2_ambiente_capac_area_estrutura',
 '5_2_ambiente_capac_area_licenciam',
 '5_2_ambiente_capac_area_edu_ambiental',
 '5_2_ambiente_capac_area_edu_amb_agric_fam',
 '5_2_ambiente_capac_area_cad_amb_rural',
 '5_2_ambiente_capac_area_resid_solidos',
 '5_2_ambiente_capac_area_prod_cons_sustent',
 '5_2_ambiente_capac_area_mudanca_cimatica',
 '5_2_ambiente_capac_area_recursos_hidr',
 '5_2_ambiente_capac_area_part_foruns_coleg',
 '5_2_ambiente_capac_area_outras',
 '5_3_ambiente_cons_existencia',
 '5_3_ambiente_cons_ano_cria',
 '5_3_ambiente_cons_formacao',
 '5_3_ambiente_cons_conselho_consultivo',
 '5_3_ambiente_cons_conselho_deliberativo',
 '5_3_ambiente_cons_c

In [28]:
#'5_4_ambiente_legis_colet_sel_existencia','5_4_ambiente_legis_colet_sel_ano',
#'5_4_ambiente_legis_san_bas_existencia','5_4_ambiente_legis_san_bas_ano',
#'5_4_ambiente_legis_gest_bachidro_existencia','5_4_ambiente_legis_gest_bachidro_ano',
#'5_4_ambiente_legis_z_prot_existencia','5_4_ambiente_legis_z_prot_ano',
#'5_4_ambiente_legis_dest_emb_agrotx_existencia','5_4_ambiente_legis_dest_emb_agrotx_ano',
#'5_4_ambiente_legis_polu_ar_existencia','5_4_ambiente_legis_polu_ar_ano',
#'5_4_ambiente_legis_perm_ext_miner_existencia','5_4_ambiente_legis_perm_ext_miner_ano',
#'5_4_ambiente_legis_fauna_silv_existencia','5_4_ambiente_legis_fauna_silv_ano',
#'5_4_ambiente_legis_floresta_existencia','5_4_ambiente_legis_floresta_ano',
#'5_4_ambiente_legis_prt_biodv_existencia','5_4_ambiente_legis_prt_biodv_ano',
#'5_4_ambiente_legis_mit_md_clima_existencia','5_4_ambiente_legis_mit_md_clima_ano',
#'5_4_ambiente_legis_nenhuma',

#'5_5_ambiente_plano_res_solido_existencia',
#'5_5_ambiente_plano_res_solido_ap_municip',

#'5_6_ambiente_par_govfed_colet_educador','5_6_ambiente_par_govfed_sala_verde','5_6_ambiente_par_govfed_circ_tela_verde',
#'5_6_ambiente_par_govfed_conf_meioambiente','5_6_ambiente_par_govfed_edu_ambiental_pgirs','5_6_ambiente_par_govfed_sust_ambiental',
#'5_6_ambiente_par_govfed_edu_ambiental_familiar','5_6_ambiente_par_govfed_confer_nacional','5_6_ambiente_par_govfed_nenhum',

#'5_7_ambiente_pag_serv_ambi_fonte_diretamente','5_7_ambiente_pag_serv_ambi_fonte_12m_municp','5_7_ambiente_pag_serv_ambi_fonte_12m_govfed',
#'5_7_ambiente_pag_serv_ambi_fonte_12m_govest','5_7_ambiente_pag_serv_ambi_fonte_12m_inipriv','5_7_ambiente_pag_serv_ambi_fonte_12m_ong',
#'5_7_ambiente_pag_serv_ambi_fonte_12m_doacoes','5_7_ambiente_pag_serv_ambi_fonte_12m_outros',
#'5_7_ambiente_pag_serv_ambi_tipo_rec_con_rec_hidrico',
#'5_7_ambiente_pag_serv_ambi_tipo_rec_con_vegt_nativa','5_7_ambiente_pag_serv_ambi_tipo_rec_con_unid_conserv','5_7_ambiente_pag_serv_ambi_tipo_rec_con_cobert_vegt',
#'5_7_ambiente_pag_serv_ambi_tipo_conserv_vegt_urbana','5_7_ambiente_pag_serv_ambi_tipo_ret_carbono','5_7_ambiente_pag_serv_ambi_tipo_outros',

'5_8_ambiente_impc_amb_ocorr_24m_existencia','5_8_ambiente_impc_amb_condicoes_extremas','5_8_ambiente_impc_amb_polu_ar','5_8_ambiente_impc_amb_polu_corpo_agua',
'5_8_ambiente_impc_amb_assoreamento','5_8_ambiente_impc_amb_diminin_vazao','5_8_ambiente_impc_amb_desmatamento','5_8_ambiente_impc_amb_queimadas',
'5_8_ambiente_impc_amb_contam_solo','5_8_ambiente_impc_amb_erosao_desert','5_8_ambiente_impc_amb_degr_areas_proteg','5_8_ambiente_impc_amb_diminin_biodv',
'5_8_ambiente_impc_amb_moradia_risco_ambiental','5_8_ambiente_impc_amb_falta_saneamento','5_8_ambiente_impc_amb_outros',

variavel = '5_8_ambiente_impc_amb_outros'
a = dfr_munic_ambiente[dfr_munic_ambiente[variavel]=='Sim'].copy()
display(a[['AM_LEGAL',variavel]].groupby(['AM_LEGAL', variavel]).size().reset_index())
display(a[['Regiao',variavel]].groupby(['Regiao', variavel]).size().reset_index())

,AM_LEGAL,5_8_ambiente_impc_amb_outros,0
0,0.00,Sim,111
1,1.00,Sim,20


,Regiao,5_8_ambiente_impc_amb_outros,0
0,1 - Norte,Sim,12
1,2 - Nordeste,Sim,41
2,3 - Sudeste,Sim,36
3,4 - Sul,Sim,27
4,5 - Centro-Oeste,Sim,15


### 0.2.6 ABA - GESTÃO DE RISCO E DESASTRES

In [29]:
# Leitura dos dados da base fornecida
dfr_munic_gst_risco = pd.read_excel('../data/MUNIC/Base_MUNIC_2020.xlsx', sheet_name='Gestão de riscos')
map_df = {'Mgrd01'   : "6_1_risco_destr_seca_exist_4anos",
          'Mgrd02'   : "6_1_risco_destr_seca_maior_impc_ano",
          'Mgrd031'  : "6_1_risco_destr_seca_maior_impc_perdas_finan",
          'Mgrd032'  : "6_1_risco_destr_seca_maior_impc_perdas_human",
          'Mgrd033'  : "6_1_risco_destr_seca_maior_impc_perdas_animais",
          'Mgrd034'  : "6_1_risco_destr_seca_maior_impc_perdas_ambient",
          'Mgrd035'  : "6_1_risco_destr_seca_maior_impc_perd_red_agricult",
          'Mgrd036'  : "6_1_risco_destr_seca_maior_impc_desertificacao",
          'Mgrd037'  : "6_1_risco_destr_seca_maior_impc_poluentes_agua",
          'Mgrd038'  : "6_1_risco_destr_seca_maior_impc_outras",
          'Mgrd039'  : "6_1_risco_destr_seca_maior_impc_nao_sabe",
          'Mgrd041'  : "6_1_risco_destr_seca_acoes_corr_cisternas",
          'Mgrd042'  : "6_1_risco_destr_seca_acoes_corr_acudes",
          'Mgrd043'  : "6_1_risco_destr_seca_acoes_corr_barragens",
          'Mgrd044'  : "6_1_risco_destr_seca_acoes_corr_pocos",
          'Mgrd045'  : "6_1_risco_destr_seca_acoes_corr_revegetacao",
          'Mgrd046'  : "6_1_risco_destr_seca_acoes_corr_agrc_adap_clima",
          'Mgrd047'  : "6_1_risco_destr_seca_acoes_corr_caminhao_pipa",
          'Mgrd048'  : "6_1_risco_destr_seca_acoes_corr_uso_sust_rec_natural",
          'Mgrd049'  : "6_1_risco_destr_seca_acoes_corr_outras",
          'Mgrd0410' : "6_1_risco_destr_seca_acoes_corr_naosabe",
          'Mgrd05'   : "6_1_risco_destr_seca_plano_contigencia",
          'Mgrd06'   : "6_2_risco_destr_alagamento_4anos_exist",
          'Mgrd07'   : "6_2_risco_destr_erosao_acel_4anos_exist",
          'Mgrd08'   : "6_3_risco_destr_enchente_4anos_exist",
          'Mgrd09'   : "6_3_risco_destr_enchente_maior_impac_ano",
          'Mgrd101'  : "6_3_risco_destr_enchente_maior_impac_edif",
          'Mgrd1011' : "6_3_risco_destr_enchente_maior_impac_edif_area",
          'Mgrd102'  : "6_3_risco_destr_enchente_maior_impac_pess",
          'Mgrd1021' : "6_3_risco_destr_enchente_maior_impac_pess_area_deslj",
          'Mgrd103'  : "6_3_risco_destr_enchente_maior_impac_obitos",
          'Mgrd1031' : "6_3_risco_destr_enchente_maior_impac_obitos_area",
          'Mgrd1041' : "6_3_risco_destr_enchente_area_nat_inund",
          'Mgrd1042' : "6_3_risco_destr_enchente_area_nao_inund",
          'Mgrd1043' : "6_3_risco_destr_enchente_area_ocp_regular",
          'Mgrd1044' : "6_3_risco_destr_enchente_area_ocp_irregul",
          'Mgrd1045' : "6_3_risco_destr_enchente_area_proc_erosiv",
          'Mgrd1046' : "6_3_risco_destr_enchente_area_outros",
          'Mgrd1047' : "6_3_risco_destr_enchente_area_naosabe",
          'Mgrd1051' : "6_3_risco_destr_enchente_acoes_corrt_barragem",
          'Mgrd1052' : "6_3_risco_destr_enchente_acoes_corrt_canais",
          'Mgrd1053' : "6_3_risco_destr_enchente_acoes_corrt_parque",
          'Mgrd1054' : "6_3_risco_destr_enchente_acoes_corrt_reserv",
          'Mgrd1055' : "6_3_risco_destr_enchente_acoes_corrt_desassor",
          'Mgrd1056' : "6_3_risco_destr_enchente_acoes_corrt_retific_rios",
          'Mgrd1057' : "6_3_risco_destr_enchente_acoes_corrt_realoc_pop",
          'Mgrd1058' : "6_3_risco_destr_enchente_acoes_corrt_revegetacao",
          'Mgrd1059' : "6_3_risco_destr_enchente_acoes_corrt_rev_rios_bacias",
          'Mgrd10510': "6_3_risco_destr_enchente_acoes_corrt_outras_solu",
          'Mgrd10511': "6_3_risco_destr_enchente_acoes_corrt_nenhuma",
          'Mgrd10512': "6_3_risco_destr_enchente_acoes_corrt_nao_sabe",
          'Mgrd11'   : "6_4_risco_destr_enxurrada_4anos_exist",
          'Mgrd12'   : "6_4_risco_destr_enxurrada_maior_impac_ano",
          'Mgrd131'  : "6_4_risco_destr_enxurrada_maior_impac_edif",
          'Mgrd1311' : "6_4_risco_destr_enxurrada_maior_impac_edif_area",
          'Mgrd132'  : "6_4_risco_destr_enxurrada_maior_impac_pess",
          'Mgrd1321' : "6_4_risco_destr_enxurrada_maior_impac_pess_area_deslj",
          'Mgrd133'  : "6_4_risco_destr_enxurrada_maior_impac_obitos",
          'Mgrd1331' : "6_4_risco_destr_enxurrada_maior_impac_obitos_area",
          'Mgrd1341' : "6_4_risco_destr_enxurrada_area_nat_inund",
          'Mgrd1342' : "6_4_risco_destr_enxurrada_area_nao_inund",
          'Mgrd1343' : "6_4_risco_destr_enxurrada_area_ocp_regular",
          'Mgrd1344' : "6_4_risco_destr_enxurrada_area_ocp_irregul",
          'Mgrd1345' : "6_4_risco_destr_enxurrada_area_proc_erosiv",
          'Mgrd1346' : "6_4_risco_destr_enxurrada_area_outros",
          'Mgrd1347' : "6_4_risco_destr_enxurrada_area_naosabe",
          'Mgrd1351' : "6_4_risco_destr_enxurrada_acoes_corrt_barragem",
          'Mgrd1352' : "6_4_risco_destr_enxurrada_acoes_corrt_canais",
          'Mgrd1353' : "6_4_risco_destr_enxurrada_acoes_corrt_parque",
          'Mgrd1354' : "6_4_risco_destr_enxurrada_acoes_corrt_reserv",
          'Mgrd1355' : "6_4_risco_destr_enxurrada_acoes_corrt_desassor",
          'Mgrd1356' : "6_4_risco_destr_enxurrada_acoes_corrt_retific_rios",
          'Mgrd1357' : "6_4_risco_destr_enxurrada_acoes_corrt_realoc_pop",
          'Mgrd1358' : "6_4_risco_destr_enxurrada_acoes_corrt_revegetacao",
          'Mgrd1359' : "6_4_risco_destr_enxurrada_acoes_corrt_rev_rios_bacias",
          'Mgrd13510': "6_4_risco_destr_enxurrada_acoes_corrt_outras_solu",
          'Mgrd13511': "6_4_risco_destr_enxurrada_acoes_corrt_nenhuma",
          'Mgrd13512': "6_4_risco_destr_enxurrada_acoes_corrt_nao_sabe",
          'Mgrd14'   : "6_5_risco_destr_deslizam_4anos_exist",
          'Mgrd15'   : "6_5_risco_destr_deslizam_maior_impac_ano",
          'Mgrd161'  : "6_5_risco_destr_deslizam_maior_impac_edif",
          'Mgrd1611' : "6_5_risco_destr_deslizam_maior_impac_edif_area",
          'Mgrd162'  : "6_5_risco_destr_deslizam_maior_impac_pess",
          'Mgrd1621' : "6_5_risco_destr_deslizam_maior_impac_pess_area_deslj",
          'Mgrd163'  : "6_5_risco_destr_deslizam_maior_impac_obitos",
          'Mgrd1631' : "6_5_risco_destr_deslizam_maior_impac_obitos_area",
          'Mgrd1641' : "6_5_risco_destr_deslizam_area_talude",
          'Mgrd1642' : "6_5_risco_destr_deslizam_area_erosao_cronico",
          'Mgrd1643' : "6_5_risco_destr_deslizam_area_sem_drenagem",
          'Mgrd1644' : "6_5_risco_destr_deslizam_area_ocp_regular",
          'Mgrd1645' : "6_5_risco_destr_deslizam_area_ocp_irregul",
          'Mgrd1646' : "6_5_risco_destr_deslizam_area_sem_ocp",
          'Mgrd1647' : "6_5_risco_destr_deslizam_area_outras_areas",
          'Mgrd1648' : "6_5_risco_destr_deslizam_area_naosabe",
          'Mgrd1651' : "6_5_risco_destr_deslizam_acoes_corrt_est_talude",
          'Mgrd1652' : "6_5_risco_destr_deslizam_acoes_corrt_retaludamento",
          'Mgrd1653' : "6_5_risco_destr_deslizam_acoes_corrt_contencao",
          'Mgrd1654' : "6_5_risco_destr_deslizam_acoes_corrt_macrodrenagens",
          'Mgrd1655' : "6_5_risco_destr_deslizam_acoes_corrt_realoc_pop",
          'Mgrd1656' : "6_5_risco_destr_deslizam_acoes_corrt_reveg_encosta",
          'Mgrd1657' : "6_5_risco_destr_deslizam_acoes_corrt_outra_solucao",
          'Mgrd1658' : "6_5_risco_destr_deslizam_acoes_corrt_nenhuma_interv",
          'Mgrd1659' : "6_5_risco_destr_deslizam_acoes_corrt_nao_sabe",
          'Mgrd171'  : "6_6_risco_destr_inst_planej_pde_enchente",
          'Mgrd172'  : "6_6_risco_destr_inst_planej_lpuos_enchente",
          'Mgrd173'  : "6_6_risco_destr_inst_planej_lei_esp_enchente",
          'Mgrd174'  : "6_6_risco_destr_inst_planej_pde_deslizamento",
          'Mgrd175'  : "6_6_risco_destr_inst_planej_lpuos_deslizamento",
          'Mgrd176'  : "6_6_risco_destr_inst_planej_lei_esp_deslizamento",
          'Mgrd177'  : "6_6_risco_destr_inst_planej_reducao_ruido",
          'Mgrd178'  : "6_6_risco_destr_inst_planej_carta_geotecnica",
          'Mgrd179'  : "6_6_risco_destr_inst_planej_plano_impln_obra",
          'Mgrd1710' : "6_6_risco_destr_inst_planej_nenhum",
          'Mgrd181'  : "6_6_risco_destr_grisco_enchente_map_area_enchente",
          'Mgrd182'  : "6_6_risco_destr_grisco_enchente_prog_hab_realocamento",
          'Mgrd183'  : "6_6_risco_destr_grisco_enchente_fiscaliz_area_risco",
          'Mgrd184'  : "6_6_risco_destr_grisco_enchente_plano_contigencia",
          'Mgrd185'  : "6_6_risco_destr_grisco_enchente_projeto_eng",
          'Mgrd186'  : "6_6_risco_destr_grisco_enchente_sistema_alerta",
          'Mgrd187'  : "6_6_risco_destr_grisco_enchente_cadastro_risco",
          'Mgrd188'  : "6_6_risco_destr_grisco_enchente_nenhum",
          'Mgrd19'   : "6_6_risco_destr_grisco_enchente_limpeza_bueiro",
          'Mgrd201'  : "6_6_risco_destr_grisco_delizam_map_area_enchente",
          'Mgrd202'  : "6_6_risco_destr_grisco_delizam_prog_hab_realocamento",
          'Mgrd203'  : "6_6_risco_destr_grisco_delizam_fiscaliz_area_risco",
          'Mgrd204'  : "6_6_risco_destr_grisco_delizam_plano_contigencia",
          'Mgrd205'  : "6_6_risco_destr_grisco_delizam_projeto_eng",
          'Mgrd206'  : "6_6_risco_destr_grisco_delizam_sistema_alerta",
          'Mgrd207'  : "6_6_risco_destr_grisco_delizam_cadastro_risco",
          'Mgrd208'  : "6_6_risco_destr_grisco_delizam_nenhum",
          'Mgrd211'  : "6_6_risco_destr_grisco_bombeiro",
          'Mgrd212'  : "6_6_risco_destr_grisco_coord_defesa_civil",
          'Mgrd213'  : "6_6_risco_destr_grisco_nucleo_defesa_civil",
          'Mgrd214'  : "6_6_risco_destr_grisco_guarda_municp",
          'Mgrd215'  : "6_6_risco_destr_grisco_nenhuma",
          'Mgrd216'  : "6_6_risco_destr_grisco_naosabe",
          'Mgrd221'  : "6_6_risco_destr_grdcivil_respns_exclusivo",
          'Mgrd222'  : "6_6_risco_destr_grdcivil_articulado",
          'Mgrd223'  : "6_6_risco_destr_grdcivil_usa_sist_desastre",
          'Mgrd224'  : "6_6_risco_destr_grdcivil_sist_proprio_desastre",
          'Mgrd225'  : "6_6_risco_destr_grdcivil_prev_orcamentaria",
          'Mgrd226'  : "6_6_risco_destr_grdcivil_outras_fontes_finan",
          'Mgrd227'  : "6_6_risco_destr_grdcivil_esp_fisc_exclusivo",
          'Mgrd228'  : "6_6_risco_destr_grdcivil_exist_viatura",
          'Mgrd229'  : "6_6_risco_destr_grdcivil_exist_pc",
          'Mgrd2210' : "6_6_risco_destr_grdcivil_exist_gps",
          'Mgrd2211' : "6_6_risco_destr_grdcivil_exist_coletes",
          'Mgrd2212' : "6_6_risco_destr_grdcivil_exist_serv_vinc_empreg",
          'Mgrd2213' : "6_6_risco_destr_grdcivil_exist_sistm_alerta_antecip",
          'Mgrd2214' : "6_6_risco_destr_grdcivil_nenhum",
          'Mgrd2215' : "6_6_risco_destr_grdcivil_naosabe",
          'Mgrd231'  : "6_6_risco_destr_protcivil_prat_educ_conscit_risco_escola",
          'Mgrd232'  : "6_6_risco_destr_protcivil_prat_educ_conscit_risco_comunid",
          'Mgrd233'  : "6_6_risco_destr_protcivil_educ_ambient_escolas",
          'Mgrd234'  : "6_6_risco_destr_protcivil_educ_ambient_comunid",
          'Mgrd235'  : "6_6_risco_destr_protcivil_campa_consct_risco_desatres",
          'Mgrd236'  : "6_6_risco_destr_protcivil_capct_prof_prtcivil_defcivil",
          'Mgrd237'  : "6_6_risco_destr_protcivil_form_nucleo_form_civil",
          'Mgrd238'  : "6_6_risco_destr_protcivil_nenhum",
          'Mgrd239'  : "6_6_risco_destr_protcivil_naosabe"
        }
dfr_munic_gst_risco = dfr_munic_gst_risco.rename(columns=map_df)
dfr_munic_gst_risco = pd.merge(dfr_munic_gst_risco, df_amz_legal, on='CodMun', how='left')

In [30]:
#dfr_munic_gst_risco.to_csv('../dados/munic/2020_munic_gst_risco_desastre.csv', index=False)

In [31]:
dfr_munic_gst_risco.columns.tolist()

['CodMun',
 'UF',
 'Cod UF',
 'Mun',
 'Faixa_pop',
 'Regiao',
 '6_1_risco_destr_seca_exist_4anos',
 '6_1_risco_destr_seca_maior_impc_ano',
 '6_1_risco_destr_seca_maior_impc_perdas_finan',
 '6_1_risco_destr_seca_maior_impc_perdas_human',
 '6_1_risco_destr_seca_maior_impc_perdas_animais',
 '6_1_risco_destr_seca_maior_impc_perdas_ambient',
 '6_1_risco_destr_seca_maior_impc_perd_red_agricult',
 '6_1_risco_destr_seca_maior_impc_desertificacao',
 '6_1_risco_destr_seca_maior_impc_poluentes_agua',
 '6_1_risco_destr_seca_maior_impc_outras',
 '6_1_risco_destr_seca_maior_impc_nao_sabe',
 '6_1_risco_destr_seca_acoes_corr_cisternas',
 '6_1_risco_destr_seca_acoes_corr_acudes',
 '6_1_risco_destr_seca_acoes_corr_barragens',
 '6_1_risco_destr_seca_acoes_corr_pocos',
 '6_1_risco_destr_seca_acoes_corr_revegetacao',
 '6_1_risco_destr_seca_acoes_corr_agrc_adap_clima',
 '6_1_risco_destr_seca_acoes_corr_caminhao_pipa',
 '6_1_risco_destr_seca_acoes_corr_uso_sust_rec_natural',
 '6_1_risco_destr_seca_acoes_corr

In [32]:
#'6_1_risco_destr_seca_exist_4anos',#'6_1_risco_destr_seca_maior_impc_ano',
#'6_1_risco_destr_seca_maior_impc_perdas_finan','6_1_risco_destr_seca_maior_impc_perdas_human',
#'6_1_risco_destr_seca_maior_impc_perdas_animais','6_1_risco_destr_seca_maior_impc_perdas_ambient','6_1_risco_destr_seca_maior_impc_perd_red_agricult',
#'6_1_risco_destr_seca_maior_impc_desertificacao','6_1_risco_destr_seca_maior_impc_poluentes_agua','6_1_risco_destr_seca_maior_impc_outras',
#'6_1_risco_destr_seca_maior_impc_nao_sabe','6_1_risco_destr_seca_acoes_corr_cisternas','6_1_risco_destr_seca_acoes_corr_acudes',
#'6_1_risco_destr_seca_acoes_corr_barragens','6_1_risco_destr_seca_acoes_corr_pocos','6_1_risco_destr_seca_acoes_corr_revegetacao',
#'6_1_risco_destr_seca_acoes_corr_agrc_adap_clima','6_1_risco_destr_seca_acoes_corr_caminhao_pipa','6_1_risco_destr_seca_acoes_corr_uso_sust_rec_natural',
#'6_1_risco_destr_seca_acoes_corr_outras','6_1_risco_destr_seca_acoes_corr_naosabe',
#'6_1_risco_destr_seca_plano_contigencia',

#'6_2_risco_destr_alagamento_4anos_exist', '6_2_risco_destr_erosao_acel_4anos_exist',

#'6_3_risco_destr_enchente_4anos_exist',#'6_3_risco_destr_enchente_maior_impac_ano',
#'6_3_risco_destr_enchente_maior_impac_edif',#'6_3_risco_destr_enchente_maior_impac_edif_area',
#'6_3_risco_destr_enchente_maior_impac_pess', #'6_3_risco_destr_enchente_maior_impac_pess_area_deslj',
#'6_3_risco_destr_enchente_maior_impac_obitos',#'6_3_risco_destr_enchente_maior_impac_obitos_area',
#'6_3_risco_destr_enchente_area_nat_inund','6_3_risco_destr_enchente_area_nao_inund', '6_3_risco_destr_enchente_area_ocp_regular',
#'6_3_risco_destr_enchente_area_ocp_irregul','6_3_risco_destr_enchente_area_proc_erosiv','6_3_risco_destr_enchente_area_outros',
#'6_3_risco_destr_enchente_area_naosabe',
#'6_3_risco_destr_enchente_acoes_corrt_barragem','6_3_risco_destr_enchente_acoes_corrt_canais',
#'6_3_risco_destr_enchente_acoes_corrt_parque','6_3_risco_destr_enchente_acoes_corrt_reserv','6_3_risco_destr_enchente_acoes_corrt_desassor',
#'6_3_risco_destr_enchente_acoes_corrt_retific_rios','6_3_risco_destr_enchente_acoes_corrt_realoc_pop','6_3_risco_destr_enchente_acoes_corrt_revegetacao',
#'6_3_risco_destr_enchente_acoes_corrt_rev_rios_bacias','6_3_risco_destr_enchente_acoes_corrt_outras_solu','6_3_risco_destr_enchente_acoes_corrt_nenhuma',
#'6_3_risco_destr_enchente_acoes_corrt_nao_sabe',

#'6_4_risco_destr_enxurrada_4anos_exist',#'6_4_risco_destr_enxurrada_maior_impac_ano',
#'6_4_risco_destr_enxurrada_maior_impac_edif',#'6_4_risco_destr_enxurrada_maior_impac_edif_area',
#'6_4_risco_destr_enxurrada_maior_impac_pess',#'6_4_risco_destr_enxurrada_maior_impac_pess_area_deslj',
#'6_4_risco_destr_enxurrada_maior_impac_obitos',#'6_4_risco_destr_enxurrada_maior_impac_obitos_area',
#'6_4_risco_destr_enxurrada_area_nat_inund','6_4_risco_destr_enxurrada_area_nao_inund','6_4_risco_destr_enxurrada_area_ocp_regular',
#'6_4_risco_destr_enxurrada_area_ocp_irregul','6_4_risco_destr_enxurrada_area_proc_erosiv','6_4_risco_destr_enxurrada_area_outros',
#'6_4_risco_destr_enxurrada_area_naosabe',
#'6_4_risco_destr_enxurrada_acoes_corrt_barragem','6_4_risco_destr_enxurrada_acoes_corrt_canais',
#'6_4_risco_destr_enxurrada_acoes_corrt_parque','6_4_risco_destr_enxurrada_acoes_corrt_reserv','6_4_risco_destr_enxurrada_acoes_corrt_desassor',
#'6_4_risco_destr_enxurrada_acoes_corrt_retific_rios','6_4_risco_destr_enxurrada_acoes_corrt_realoc_pop','6_4_risco_destr_enxurrada_acoes_corrt_revegetacao',
#'6_4_risco_destr_enxurrada_acoes_corrt_rev_rios_bacias','6_4_risco_destr_enxurrada_acoes_corrt_outras_solu','6_4_risco_destr_enxurrada_acoes_corrt_nenhuma',
#'6_4_risco_destr_enxurrada_acoes_corrt_nao_sabe',

#'6_5_risco_destr_deslizam_4anos_exist',#'6_5_risco_destr_deslizam_maior_impac_ano',
#'6_5_risco_destr_deslizam_maior_impac_edif',#'6_5_risco_destr_deslizam_maior_impac_edif_area',
#'6_5_risco_destr_deslizam_maior_impac_pess',#'6_5_risco_destr_deslizam_maior_impac_pess_area_deslj',
#'6_5_risco_destr_deslizam_maior_impac_obitos',#'6_5_risco_destr_deslizam_maior_impac_obitos_area',
#'6_5_risco_destr_deslizam_area_talude','6_5_risco_destr_deslizam_area_erosao_cronico','6_5_risco_destr_deslizam_area_sem_drenagem',
#'6_5_risco_destr_deslizam_area_ocp_regular','6_5_risco_destr_deslizam_area_ocp_irregul','6_5_risco_destr_deslizam_area_sem_ocp',
#'6_5_risco_destr_deslizam_area_outras_areas','6_5_risco_destr_deslizam_area_naosabe',
#'6_5_risco_destr_deslizam_acoes_corrt_est_talude', '6_5_risco_destr_deslizam_acoes_corrt_retaludamento','6_5_risco_destr_deslizam_acoes_corrt_contencao',
#'6_5_risco_destr_deslizam_acoes_corrt_macrodrenagens','6_5_risco_destr_deslizam_acoes_corrt_realoc_pop','6_5_risco_destr_deslizam_acoes_corrt_reveg_encosta',
#'6_5_risco_destr_deslizam_acoes_corrt_outra_solucao','6_5_risco_destr_deslizam_acoes_corrt_nenhuma_interv','6_5_risco_destr_deslizam_acoes_corrt_nao_sabe',

'6_6_risco_destr_inst_planej_pde_enchente','6_6_risco_destr_inst_planej_lpuos_enchente','6_6_risco_destr_inst_planej_lei_esp_enchente',
'6_6_risco_destr_inst_planej_pde_deslizamento','6_6_risco_destr_inst_planej_lpuos_deslizamento','6_6_risco_destr_inst_planej_lei_esp_deslizamento',
'6_6_risco_destr_inst_planej_reducao_ruido','6_6_risco_destr_inst_planej_carta_geotecnica','6_6_risco_destr_inst_planej_plano_impln_obra',
'6_6_risco_destr_inst_planej_nenhum',
'6_6_risco_destr_grisco_enchente_map_area_enchente','6_6_risco_destr_grisco_enchente_prog_hab_realocamento','6_6_risco_destr_grisco_enchente_fiscaliz_area_risco',
'6_6_risco_destr_grisco_enchente_plano_contigencia','6_6_risco_destr_grisco_enchente_projeto_eng','6_6_risco_destr_grisco_enchente_sistema_alerta',
'6_6_risco_destr_grisco_enchente_cadastro_risco','6_6_risco_destr_grisco_enchente_nenhum','6_6_risco_destr_grisco_enchente_limpeza_bueiro',

'6_6_risco_destr_grisco_delizam_map_area_enchente','6_6_risco_destr_grisco_delizam_prog_hab_realocamento','6_6_risco_destr_grisco_delizam_fiscaliz_area_risco',
'6_6_risco_destr_grisco_delizam_plano_contigencia','6_6_risco_destr_grisco_delizam_projeto_eng','6_6_risco_destr_grisco_delizam_sistema_alerta',
'6_6_risco_destr_grisco_delizam_cadastro_risco','6_6_risco_destr_grisco_delizam_nenhum',

'6_6_risco_destr_grisco_bombeiro','6_6_risco_destr_grisco_coord_defesa_civil',
'6_6_risco_destr_grisco_nucleo_defesa_civil','6_6_risco_destr_grisco_guarda_municp','6_6_risco_destr_grisco_nenhuma','6_6_risco_destr_grisco_naosabe',

variavel = '6_6_risco_destr_grisco_naosabe'
a = dfr_munic_gst_risco[dfr_munic_gst_risco[variavel]=='Sim'].copy()
display(a[['AM_LEGAL',variavel]].groupby(['AM_LEGAL', variavel]).size().reset_index())
display(a[['Regiao',variavel]].groupby(['Regiao', variavel]).size().reset_index())

,AM_LEGAL,6_6_risco_destr_grisco_naosabe,0
0,0.00,Sim,89
1,1.00,Sim,31


,Regiao,6_6_risco_destr_grisco_naosabe,0
0,1 - Norte,Sim,19
1,2 - Nordeste,Sim,43
2,3 - Sudeste,Sim,26
3,4 - Sul,Sim,12
4,5 - Centro-Oeste,Sim,20


In [33]:
dfr_munic_gst_risco['.head()']

KeyError: '.head()'